# NanoChat Easy - Mid Training



## Import model and tokenizer

In [16]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, get_cosine_schedule_with_warmup


model_id = "karpathy/nanochat-d32"
revision = "refs/pr/1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=revision,
    torch_dtype=torch.bfloat16 if device.type == "cuda" else torch.float32,
).to(device)

## Setup LoRA

In [17]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=2,
    lora_dropout=0.00,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "fc1", "fc2"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 1,880,227,840 || trainable%: 0.0627


In [18]:
model.modules

<bound method Module.modules of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): NanoChatForCausalLM(
      (model): NanoChatModel(
        (initial_norm): NanoChatRMSNorm(eps=1e-06)
        (layers): ModuleList(
          (0-31): 32 x NanoChatDecoderLayer(
            (self_attn): NanoChatAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=1, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

## Demo the model

In [19]:
print("=" * 80)
print("TEST 1: Plain Autoregressive Prompt")
print("=" * 80)
prompt = "The Eiffel Tower stands in Paris and"
test_inputs = tokenizer(prompt, return_tensors="pt").to(device)


with torch.no_grad():
    test_outputs = model.generate(
        **test_inputs,
        max_new_tokens=64,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
    )

generated_tokens = test_outputs[0, test_inputs["input_ids"].shape[1] :]
print(f"Prompt: {prompt}")
print(f"\nGenerated: {tokenizer.decode(generated_tokens, skip_special_tokens=True)}")
print("=" * 80)

TEST 1: Plain Autoregressive Prompt
Prompt: The Eiffel Tower stands in Paris and

Generated:  is one of the most famous landmarks in the world. It is located on the Champ de Mars in the heart of the city. The tower was built for the 1889 World's Fair. It was designed by the French engineer Gustave Eiffel and took 2 years to build. The Eiffel Tower stands 324 meters


In [20]:
print("=" * 80)
print("TEST 2: Chat Template")
print("="*80)
conversation = [
    {"role": "user", "content": "What is the capital of France?"},
]

inputs = tokenizer.apply_chat_template(
    conversation, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
).to(device)

print(f"Formatted prompt: {tokenizer.decode(inputs['input_ids'][0])}")
print(f"Input IDs: {inputs['input_ids'][0].tolist()}")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False
    )

generated_tokens = outputs[0, inputs["input_ids"].shape[1] :]
print(f"\nGenerated: {tokenizer.decode(generated_tokens)}")
print("=" * 80)

TEST 2: Chat Template
Formatted prompt: <|bos|><|user_start|>What is the capital of France?<|user_end|><|assistant_start|>
Input IDs: [65527, 65528, 1442, 309, 261, 3429, 281, 4215, 63, 65529, 65530]

Generated: The capital of France is Paris.<|assistant_end|>


## Dataset

In [8]:
raw_dataset = load_dataset("HuggingFaceTB/smoltalk2", "Mid", split="OpenThoughts3_1.2M")
splits = raw_dataset.train_test_split(test_size=0.05, seed=13)
train_dataset = splits["train"]
eval_dataset = splits["test"]

Generating Llama_Nemotron_Post_Training_Dataset_reasoning_r1 split: 100%|██████████| 3644790/3644790 [02:07<00:00, 28616.91 examples/s]
Generating OpenThoughts3_1.2M split:  12%|█▏        | 135598/1135104 [00:15<01:44, 9528.89 examples/s]

### Process the Dataset

In [24]:
max_length = 2048
max_train_examples = 20000
max_eval_examples = 128

def flatten_messages(messages):
    lines = []
    for msg in messages:
        role = msg.get("role", "")
        content = msg.get("content", "")
        if role:
            lines.append(f"{role.upper()}: {content}")
        else:
            lines.append(content)
    return "\n".join(lines)

def tokenize_example(example):
    text = flatten_messages(example["messages"])
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding=False,
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
    }


if max_train_examples is not None:
    train_dataset = train_dataset.select(range(min(len(train_dataset), max_train_examples)))
    train_dataset = train_dataset.map(tokenize_example, remove_columns=train_dataset.column_names)
if max_eval_examples is not None:
    eval_dataset = eval_dataset.select(range(min(len(eval_dataset), max_eval_examples)))
    eval_dataset = eval_dataset.map(tokenize_example, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map: 100%|██████████| 128/128 [00:02<00:00, 45.46 examples/s]


## Training Configuration

In [27]:
train_batch_size = 1
eval_batch_size = 1
num_epochs = 1
gradient_accumulation_steps = 16
learning_rate = 6e-5
weight_decay = 0.01
adam_beta1 = 0.9
adam_beta2 = 0.95
warmup_ratio = 0.02
logging_frequency = 20

## Create a `DataLoader` 👴

In [30]:
def collate_fn(batch):
    batch_dict = {
        "input_ids": [record["input_ids"] for record in batch],
        "attention_mask": [record["attention_mask"] for record in batch],
    }
    padded = tokenizer.pad(batch_dict, padding=True, return_tensors="pt")
    labels = padded["input_ids"].clone()
    labels[padded["attention_mask"] == 0] = -100
    padded["labels"] = labels
    return padded


TrainLoader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)
EvalLoader = DataLoader(eval_dataset, batch_size=eval_batch_size, shuffle=False, collate_fn=collate_fn)


## Optimizer

In [31]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=weight_decay,
)

# Learning Rate Scheduler

In [32]:
num_update_steps_per_epoch = max(len(TrainLoader) // gradient_accumulation_steps, 1)
max_train_steps = num_epochs * num_update_steps_per_epoch
warmup_steps = max(1, int(max_train_steps * warmup_ratio))
scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, max_train_steps)

# The Training Loop

In [ ]:

model.train()
global_step = 0
running_loss = 0.0
running_steps = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    optimizer.zero_grad(set_to_none=True)
    for step, batch in enumerate(TrainLoader, start=1):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        running_loss += outputs.loss.float().item()
        running_steps += 1

        if step % gradient_accumulation_steps == 0 or step == len(TrainLoader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)
            global_step += 1

            if global_step % logging_frequency == 0:
                current_lr = scheduler.get_last_lr()[0]
                mean_loss = running_loss / running_steps
                print(f"step={global_step:05d} | loss={mean_loss:.4f} | lr={current_lr:.2e}")
                running_loss = 0.0
                running_steps = 0

    train_loss = running_loss / running_steps if running_steps > 0 else float("nan")
    print(f"Training loss after epoch {epoch + 1}: {train_loss:.4f}")
    
    model.eval()
    losses = []
    with torch.no_grad():
        for _, batch in enumerate(EvalLoader, start=1):
            batch = {key: value.to(device) for key, value in batch.items()}
            loss = model(**batch).loss
            losses.append(loss.float().item())
    model.train()
    val_loss = sum(losses) / len(losses) if losses else float("nan")
        
    print(f"Validation loss after epoch {epoch + 1}: {val_loss:.4f}")

print("Training complete.")

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 1/1
step=00020 | loss=1.7836 | lr=4.80e-05
step=00040 | loss=1.7800 | lr=6.00e-05
step=00060 | loss=1.8128 | lr=5.99e-05
step=00080 | loss=1.7713 | lr=5.97e-05
step=00100 | loss=1.7851 | lr=5.94e-05
